In [1]:
import pandas as pd
from utility import get_similar_users_avg_rating, merge_df

In [ ]:
train_df = pd.read_csv('data/user_concat_text_10_train.csv', sep=';')

In [ ]:
df_filtered = pd.read_csv("data/english_reviews.csv")
df_cleaned = merge_df(df_filtered)

In [ ]:
df_cleaned['bert_rating'] = df_cleaned.apply(lambda row: get_similar_users_avg_rating(
    train_df, df_filtered, row['user_id'], row['gmap_id']), axis=1)

# Replace None with pd.nan in 'bert_rating' column
df_cleaned['bert_rating'].replace({None: pd.nan}, inplace=True)